In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
data = pd.read_csv('../raw_data/state_sightings.csv', low_memory=False)

In [3]:
data.head()

,datetime,city,state,shape,duration (seconds),state_name,state_population,state_military_bases,state_cities,day,month,month_number,year_number,season,year_season
0,1910-01-01 00:00:00,kirksville (near),mo,disk,120.0,Missouri,7474758.0,2.0,1020.0,1910-01-01,1910-01,1,1910,winter,1910-winter
1,1910-06-01 15:00:00,wills point,tx,cigar,120.0,Texas,33717239.0,5.0,1679.0,1910-06-01,1910-06,6,1910,summer,1910-summer
2,1920-06-11 21:00:00,cicero,in,unknown,60.0,Indiana,6236667.0,1.0,677.0,1920-06-11,1920-06,6,1920,summer,1920-summer
3,1925-12-28 18:00:00,atkinson (6 miles north of),il,disk,60.0,Illinois,18104498.0,1.0,1362.0,1925-12-28,1925-12,12,1925,winter,1925-winter
4,1929-07-05 14:00:00,buchanan (or burns),or,disk,60.0,Oregon,5311863.0,0.0,373.0,1929-07-05,1929-07,7,1929,summer,1929-summer


In [4]:
def value_count_func (series_):
    return len(series_.value_counts())

def most_freq_func (series_):
    x_ = series_.value_counts()
    return list(x_.keys())[0]

def state_season_df (data_, state_):
    
    data_state = data_[data_['state'] == state_]
    
    data_state_season = data_state.groupby(by='year_season').agg({'datetime':'count','duration (seconds)':'mean',
                                          'city':value_count_func,'shape':most_freq_func,'day':value_count_func,
                                          'state_population':'first', 'state_military_bases':'first','state_cities':'first'})
    
    data_state_season.rename(columns={'datetime':'sightings_period','duration (seconds)':'avg_duration(seconds)',
                                  'city':'sightings_cities','day':'sightings_days_period'},inplace=True)
    
    data_state_season['state'] = state_
    
    return data_state_season.reset_index().set_index('state')

def previous_sightings (x_, idx_):
    
    row_index_state, row_index_year, row_index_season = idx_.split('-') 
    
    period_1 = f"{row_index_state}-{int(row_index_year)-1}-{row_index_season}"
    period_2 = f"{row_index_state}-{int(row_index_year)-2}-{row_index_season}"
    period_3 = f"{row_index_state}-{int(row_index_year)-3}-{row_index_season}"

    try:
        res_1 = x_.loc[period_1].sightings_days_period
    except:
        res_1 = 0
        
    try:
        res_2 = x_.loc[period_2].sightings_days_period
    except:
        res_2 = 0
        
    try:
        res_3 = x_.loc[period_3].sightings_days_period
    except:
        res_3 = 0
    
    return res_1, res_2, res_3    

In [5]:
STATES_ = list(data.state.unique()) 

x_ = pd.DataFrame()

for st_ in STATES_:
    
    y_ = state_season_df (data, st_)
    x_ = pd.concat([x_,y_])
    
x_ = x_.reset_index()

x_['state_season'] = x_['state']+ '-' + x_['year_season']

# x_.drop(columns=['year_season','state'], inplace=True)

x_.set_index('state_season',inplace=True)

x_['sigthing_days_1_year_before'] = None
x_['sigthing_days_2_year_before'] = None
x_['sigthing_days_3_year_before'] = None

In [ ]:
indexes_ = list(x_.index)

for i in indexes_:
    
    res1, res2, res3 = previous_sightings (x_, i)
    
    x_.loc[i,'sigthing_days_1_year_before'] = res1
    x_.loc[i,'sigthing_days_2_year_before'] = res2
    x_.loc[i,'sigthing_days_3_year_before'] = res3


In [ ]:
x_[x_['state'] == 'tx']

In [ ]:
x_.to_csv('../raw_data/features_by_state_season.csv',header=True,index=False)